some helper functions

In [314]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
def safe_div_s(a, b, b_min = 0.01):
    if (b > 0 or b < 0):
        return a / b
    else:
        return a / b_min
def safe_div(df, dividend_name, divisor_name, b_min = 0.01): #a safe division of two columns
    try:
        df[divisor_name]
    except:
        print ('Warning: A column name with the name must be specified =' + divisor_name)
        return None
    try:
        df[dividend_name]
    except:
        print ('Warning: A column name with the name must be specified =' + dividend_name)
        return None
    df.loc[df[divisor_name] == 0, divisor_name] = b_min
    df[dividend_name[:7]+'/'+divisor_name[:7]] = df[dividend_name]/df[divisor_name]
    return df
def median_calculated(df, filter_column, target_column): #it calculates a list of median values filterd by filter_column
    try:
        df[filter_column]
    except:
        print ('Warning: A column name with the name must be specified =' + filter_column)
        return None
    try:
        df[target_column]
    except:
        print ('Warning: A column name with the name must be specified =' + target_column)
        return None
    my_list = df[filter_column].unique()
    #print(my_list)
    #print(df.loc[df[filter_column] == my_list[0]])
    result = []
    for el in my_list:
        #print(df.loc[df[filter_column] == el])
        #print(df.loc[df[filter_column] == el][target_column])
        result.append(df.loc[df[filter_column] == el][target_column].median())
    return result
# it calculates dividend/divisor, if divisor =0, it repaces it on median calculated with all rows in range by filter_columns
#a = [[5, 1.2, 4.2], [1, 0, 0.03], [8, 5, 1], [8,2,4], [16,4,4], [32,9,4], [32,0,4], [-9,0,2]]
#df = pd.DataFrame(a, columns=['dividend', 'diviser', 'filter'])
#print(df)
#print(median_safe_div(df, 'dividend', 'diviser', 'filter'))
#   dividend  diviser  filter
#0         5      1.2    4.20
#1         1      0.0    0.03
#2         8      5.0    1.00
#3         8      2.0    4.00
#4        16      4.0    4.00
#5        32      9.0    4.00
#6        32      0.0    4.00
#7        -9      0.0    2.00
#   dividend  diviser  filter  dividend/diviser
#0         5      1.2    4.20          4.166667
#1         1      0.0    0.03          4.000000
#2         8      5.0    1.00          1.600000
#3         8      2.0    4.00          4.000000
#4        16      4.0    4.00          4.000000
#5        32      9.0    4.00          3.555556
#6        32      0.0    4.00          4.000000
#7        -9      0.0    2.00          4.000000



def div_median(df, dividend_name, divisor_name, filter_column):
    try:
        df[divisor_name]
    except:
        print ('Warning: A column name with the name must be specified =' + divisor_name)
        return None
    try:
        df[dividend_name]
    except:
        print ('Warning: A column name with the name must be specified =' + dividend_name)
        return None
    try:
        df[filter_column]
    except:
        print ('Warning: A column name with the name must be specified =' + filter_column)
        return None
    my_list = df[filter_column].unique()
    #calculate basic
    new_name = dividend_name[:8]+'/'+divisor_name[:8]
    df[new_name] = df[dividend_name]/df[divisor_name]
    median =df[new_name].median()
    median_list = median_calculated(df, filter_column, new_name)
    j=0
    #replacing infinity on median of every range (by filter_column)
    for el in median_list:
        if (el == np.inf or el == -np.inf):
            el = df[new_name].median()
            median_list[j] = df[new_name].median()
        j+=1    

    i=0
    #if range = 1 row and it had 0 in diviser --> replace result column value on median of entire column
    for el in my_list:
        df.loc[(df[filter_column] == el)&((df[new_name]== np.inf) | (df[new_name]== -np.inf)), new_name] = median_list[i]
        i+=1
    #df[new_name].replace(to_replace=np.inf, value=median, inplace=True)
    #df[new_name].replace(to_replace=-np.inf, value=median, inplace=True)
    return (df)


main_file_path = 'train.csv'
data = pd.read_csv(main_file_path)
pd.set_option('display.max_rows', 100)
#print(median_calculated(data, 'Neighborhood', 'LotArea'))
#print(median_calculated(df, 'one', 'two'))
data = div_median(data, 'SalePrice', 'LotArea', 'Neighborhood')
#data_g_p = data['SalePric/LotArea']
#print(data_g_p)
#print(data_g_p)
#print(data)
#print(data.describe())
#print(data.columns)
unique_data = data['Neighborhood'].unique()
#print(unique_data)
#neighborhood = pd.get_dummies(data['Neighborhood'], columns=['Neighborhood'])
data_n = pd.get_dummies(data, columns=['Neighborhood'])
only_neighborhood_col = list(set(data_n.columns)-set(data.columns))
#print(neighborhood)
price_columns = ['SalePric/LotArea']
#data_sparce = pd.concat([neighborhood, data_g_p], axis=1)
#print(result)
#price_lot_data = safe_div(data['SalePrice'], data['LotArea'], 1)
#print(price_lot_data)

In [315]:
#y = data_sparce['SalePric/LotArea']
data_n['allarea'] = data_n['1stFlrSF']+data_n['2ndFlrSF']+data_n['BsmtFinSF1']
y_col = ['SalePric/LotArea', 'SalePrice']
y = data_n[y_col]
dummy_data = data_n[only_neighborhood_col] 
x= data_n
#print(x.head())
#print(y.head())
train_x, val_x, train_y, val_y = train_test_split(x, y,random_state = 100)

In [316]:
# Define model
helper_model = LinearRegression()

# Fit model
helper_model.fit(train_x[only_neighborhood_col], train_y['SalePric/LotArea'])
print(train_x)
print(train_x[only_neighborhood_col])
train_x[e_price]=helper_model.predict(train_x[only_neighborhood_col])
print(helper_model.predict(train_x[only_neighborhood_col]))

soil_prices = pd.DataFrame(helper_model.predict(train_x[only_neighborhood_col]))
predicted_soil_prices = pd.DataFrame(helper_model.predict(val_x[only_neighborhood_col]))
#predicted_soil_prices=pd.DataFrame(helper_model.predict(val_x)) 
#actual_soil_prices=val_y
#mean_absolute_error(actual_soil_prices, predicted_soil_prices)
#print(type(predicted_soil_prices))
#print(type(data_n['LotArea']))
#print(type(aaa))
#print(aaa.head())
train_x['lot_price'] = soil_prices[0]*train_x['LotArea']
val_x['lot_price'] = predicted_soil_prices[0]*val_x['LotArea']
print(soil_prices[0])
print(train_x['lot_price'])
print(soil_prices[0]*train_x['LotArea'])
#print(train_x[predictors])
#print(train_x['LotArea'].head())
#print(train_x['lot_price'].head())
#print(train_x['SalePrice'].head())
#data['lot_price'] = helper_model.predict(train_x)*data_n['LotArea']
#print(predicted_soil_prices)
#print(actual_soil_prices.head)
#data_comaparasion = pd.DataFrame(np.concatenate([actual_soil_prices,  predicted_soil_prices], axis=0))
#print(data_comaparasion)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
774    775          20       RL      110.000    14226   Pave   NaN      Reg   
92      93          30       RL       80.000    13360   Pave  Grvl      IR1   
360    361          85       RL          nan     7540   Pave   NaN      IR1   
549    550          60       FV       75.000     9375   Pave   NaN      Reg   
546    547          50       RL       70.000     8737   Pave   NaN      IR1   
1034  1035          30       RL       50.000     6305   Pave   NaN      Reg   
4        5          60       RL       84.000    14260   Pave   NaN      IR1   
717    718          20       RL       80.000    10000   Pave   NaN      Reg   
1001  1002          30       RL       60.000     5400   Pave   NaN      Reg   
176    177          60       RL       97.000    10029   Pave   NaN      IR1   
1143  1144          20       RL          nan     9000   Pave   NaN      Reg   
1211  1212          50       RL      152.000    1213

NameError: name 'e_price' is not defined

In [ ]:
predictors=['allarea', 'GarageCars', 'BedroomAbvGr', 'OverallQual', 'OverallCond','FullBath', 'lot_price', 'LotArea']
model = DecisionTreeRegressor()
print(train_x[predictors])
#model.fit(train_x[predictors], train_y['SalePrice'])
